In [1]:
import xml.etree.cElementTree as et
import pandas as pd
import os
import re
from tqdm import notebook
import ast

## Notarieel archief

In [12]:
df_cols = ["uuid", "rubriek", "notaris", "inventarisNr", "akteNr", "akteType", 
           "datering", "taal", "beschrijving", "namen", 'urls']
df = pd.DataFrame(columns = df_cols)
for file in notebook.tqdm(["SAA_Index_op_notarieel_archief_20191105_001", 
             "SAA_Index_op_notarieel_archief_20191105_002",
             "SAA_Index_op_notarieel_archief_20191105_003",
             "SAA_Index_op_notarieel_archief_20191105_004",
             "SAA_Index_op_notarieel_archief_20191105_005",
             "SAA_Index_op_notarieel_archief_20191105_006",
             "SAA_Index_op_notarieel_archief_20191105_007",
             "SAA_Index_op_notarieel_archief_20191105_008",
             "SAA_Index_op_notarieel_archief_20191105_009",
             "SAA_Index_op_notarieel_archief_20191105_010"]):
    parsedXML = et.parse("../notarieel/" + file + ".xml")
    root = parsedXML.getroot()
    rows = []

    for node in root: 
        if node is not None:
            s_uuid = node.find("uuid").text if node.find("uuid") is not None else None
            s_rubriek = node.find("rubriek").text if node.find("rubriek") is not None else None
            s_notaris = node.find("notaris").text if node.find("notaris") is not None else None
            s_inventarisNr = node.find("inventarisNr").text if node.find("inventarisNr") is not None else None
            s_akteNr= node.find("akteNr").text if node.find("akteNr") is not None else None
            s_akteType = node.find("akteType").text if node.find("akteType") is not None else None
            s_datering = node.find("datering").text if node.find("datering") is not None else None
            s_taal = node.find("taal").text if node.find("taal") is not None else None
            s_beschrijving = node.find("beschrijving").text if node.find("beschrijving") is not None else None
            s_namen = []
            if node.find("persoonsnamen") is not None:
                namen = []
                for x in node.find("persoonsnamen"):
                    voornaam = x.find("voornaam").text if x.find("voornaam") is not None else None
                    tussenvoegsel = x.find("tussenvoegsel").text if x.find("tussenvoegsel") is not None else None
                    achternaam = x.find("achternaam").text if x.find("achternaam") is not None else None
                    scanNaam = x.find("scanNaam").text if x.find("scanNaam") is not None else None
                    scanPositie = x.find("scanPositie").text if x.find("scanPositie") is not None else None
                    uuidNaam = x.find("uuidNaam").text if x.find("uuidNaam") is not None else None

                    namen.append({"voornaam": voornaam, "tussenvoegsel": tussenvoegsel, 
                         "achternaam": achternaam, "scanNaam": scanNaam, 
                         "scanPositie": scanPositie, "uuidNaam": uuidNaam})
            else:
                namen = None
            if node.find("urlScans") is not None:
                urls = []
                for x in node.find("urlScans"):
                    urlScan = x.text if x is not None else None
                    if urlScan != None:
                        urls.append(re.search(r'#.+', urlScan).group(0)[1:])
            else:
                urls = None
        else:
            s_uuid = None
            s_rubriek = None
            s_notaris = None
            s_inventarisNr = None
            s_akteNr= None
            s_akteType = None
            s_datering = None
            s_taal = None
            s_beschrijving = None
        rows.append({"uuid": s_uuid, "rubriek": s_rubriek, 
                     "notaris": s_notaris, "inventarisNr": s_inventarisNr, 
                     "akteNr": s_akteNr, "akteType": s_akteType,
                     "datering": s_datering, "taal": s_taal,
                     "beschrijving": s_beschrijving, "namen": namen, "urls":urls})

    out_df = pd.DataFrame(rows, columns = df_cols)
    df = df.append(out_df)
    df = df.reset_index(drop=True)


In [47]:
df

uuid rubriek  \
0       06306e00-98da-1130-ce03-93cbc225e77c       2   
1       122902ff-2633-2d1d-1fdd-0842fd195e3f       2   
2       22c4dd27-eec8-02bb-33d9-c8fea3161d18       2   
3       2b017209-8640-4307-1423-df64615a59f6       2   
4       30611e33-7c57-2b2a-bb0e-41227699255c       2   
...                                      ...     ...   
292361  4d0a5598-5c6a-7040-e053-b784100a6264     507   
292362  4d0a5598-6fb6-7040-e053-b784100a6264     507   
292363  4d0a5598-8a27-7040-e053-b784100a6264     507   
292364  4d0a5598-8a45-7040-e053-b784100a6264     507   
292365  4d0a5598-8a4f-7040-e053-b784100a6264     507   

                               notaris inventarisNr  akteNr    akteType  \
0                       ELLERT DE VEER            6  160222   Testament   
1                       ELLERT DE VEER            6  162623  Insinuatie   
2                       ELLERT DE VEER            6  165855  Machtiging   
3                       ELLERT DE VEER            6  167432   Testament   
4                       ELLERT DE VEER            6  168465  Machtiging   
...                                ...          ...     ...         ...   
292361  Mr. AUGUST HENDRIK DE MELANDER       20034B   53690      Overig   
292362  Mr. AUGUST HENDRIK DE MELANDER       20034B   54446        Koop   
292363  Mr. AUGUST HENDRIK DE MELANDER       20034B   55431   Testament   
292364  Mr. AUGUST HENDRIK DE MELANDER       20034B   55436  Machtiging   
292365  Mr. AUGUST HENDRIK DE MELANDER       20034B   55438  Machtiging   

          datering        taal                          beschrijving  \
0       1587-03-18  nederlands                          \nhandmerk\n   
1       1586-10-02  nederlands                                  None   
2       1589-01-05  nederlands                                  None   
3       1588-06-06  nederlands  \nCodicil. Legaat aan dienstmaagd.\n   
4       1587-06-23  nederlands                                  None   
...            ...         ...                                   ...   
292361  1818-05-26  nederlands                                  None   
292362  1818-01-05  nederlands                                  None   
292363  1818-06-23  nederlands                                  None   
292364  1818-06-30  nederlands                                  None   
292365  1818-07-04  nederlands                                  None   

                                                    namen  \
0       [{'voornaam': 'Elisabeth', 'tussenvoegsel': 'd...   
1       [{'voornaam': 'Aert', 'tussenvoegsel': None, '...   
2       [{'voornaam': 'Feycke', 'tussenvoegsel': None,...   
3       [{'voornaam': 'Pieter', 'tussenvoegsel': None,...   
4       [{'voornaam': 'Bartholomeus', 'tussenvoegsel':...   
...                                                   ...   
292361  [{'voornaam': 'Johannes Daniel', 'tussenvoegse...   
292362  [{'voornaam': 'Hendrik', 'tussenvoegsel': None...   
292363  [{'voornaam': 'Anke', 'tussenvoegsel': None, '...   
292364  [{'voornaam': 'Trijntje', 'tussenvoegsel': Non...   
292365  [{'voornaam': 'Johan Joachim', 'tussenvoegsel'...   

                                                     urls  
0                                   [KLAC00444000031.JPG]  
1                                   [KLAC00444000021.JPG]  
2                                   [KLAC00444000094.JPG]  
3              [KLAC00444000087.JPG, KLAC00444000088.JPG]  
4              [KLAC00444000051.JPG, KLAC00444000052.JPG]  
...                                                   ...  
292361  [KLAB02845000267.JPG, KLAB02845000268.JPG, KLA...  
292362  [KLAB02845000004.JPG, KLAB02845000005.JPG, KLA...  
292363         [KLAB02845000313.JPG, KLAB02845000314.JPG]  
292364         [KLAB02845000318.JPG, KLAB02845000319.JPG]  
292365         [KLAB02845000322.JPG, KLAB02845000323.JPG]  

[292366 rows x 11 columns]

## Scanned Data

In [8]:
def namespace(element):
    m = re.match(r'\{.*\}', element.tag)
    return m.group(0) if m else ''

In [9]:
files = []
directory = 'D:\Documenten\Studie\Master\Scriptie\scanned\page'
for filename in os.listdir(directory):
    if filename.endswith(".xml"):
        files.append(filename)
    else:
        continue

In [10]:
df_cols = ["page", "textregion"]
scan_df = pd.DataFrame(columns = df_cols)
for file in tqdm_notebook(files):
    parsedXML = et.parse('D:/Documenten/Studie/Master/Scriptie/scanned/page/' + file)
    root = parsedXML.getroot()
    rows = []
    page = file[:-3] + "JPG"
    text = ""
    for node in root: 
        nspace = namespace(node)
        if node is not None:
            for child in node:
                if child.find(nspace + "TextEquiv") is not None:
                    if child.find(nspace + "TextEquiv").find(nspace + "Unicode").text is not None:
                        text += str(child.find(nspace + "TextEquiv").find(nspace + "Unicode").text)
    if text != "":           
        rows.append({"page": page, "textregion": text })
        out_df = pd.DataFrame(rows, columns = df_cols)
        scan_df = scan_df.append(out_df)
        scan_df = scan_df.reset_index(drop=True)


NameError: name 'tqdm_notebook' is not defined

## Cleaning data

In [42]:
def cleaner(string):
    string  = re.sub('\\n', ' ', string)
    return string

In [51]:
notebook.tqdm.pandas()

D:\Anaconda\lib\site-packages\tqdm\std.py:651: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [44]:
scan_df.textregion = scan_df.textregion.progress_apply(cleaner)

In [45]:
def writer(row):
    file = open(str(row['page']) + ".txt", "w") 
    file.write(row['textregion']) 
    file.close() 

In [49]:
os.chdir('../text_files')
scan_df.progress_apply(writer, axis=1)

0        None
1        None
2        None
3        None
4        None
         ... 
43177    None
43178    None
43179    None
43180    None
43181    None
Length: 43182, dtype: object

## Clean Dataset

In [41]:
df_cols = ["page", "text"]
scan_df = pd.DataFrame(columns = df_cols)
for filename in notebook.tqdm(os.listdir('../clean')):
    if filename.endswith(".JPG.ticcl.folia.xml"): 
        parsedXML = et.parse('../clean/' + filename)
        root = parsedXML.getroot()
        rows = []
        page = filename[:-16]
        text = ""
        for node in root: 
            nspace = namespace(node)
            if node is not None:
                if node.find(nspace + 'p') is not None:
                    child = node.find(nspace + 'p')
                    if child.find(nspace + 't') is not None:
                        text += child.find(nspace + 't').text
        if text != "":           
            rows.append({"page": page, "text": text })
            out_df = pd.DataFrame(rows, columns = df_cols)
            scan_df = scan_df.append(out_df)
            scan_df = scan_df.reset_index(drop=True)

    else:
        continue
scan_df = scan_df.set_index('page')

In [14]:
jan_verleij = df[df.notaris == "JAN VERLEIJ"].reset_index(drop=True)

In [104]:
def merge(row):
    text = ""
    for x in row.urls:
        if x in scan_df.index:
            text += scan_df.text[x]
        else:
            print(row)
    if text == "":
        text = None
    return text

In [105]:
jan_verleij['text'] = jan_verleij.progress_apply(merge, axis=1)

uuid                         1dd97aef-0f5a-b513-496c-0d81dba0922d
rubriek                                                       358
notaris                                               JAN VERLEIJ
inventarisNr                                                11900
akteNr                                                      13002
akteType                                         Boedelinventaris
datering                                               1747-01-04
taal                                                   nederlands
beschrijving                                                 None
namen           [{'voornaam': 'Margareta', 'tussenvoegsel': No...
urls            [KLAB05523000014.JPG, KLAB05523000015.JPG, KLA...
Name: 224, dtype: object
uuid                         41ff8d5b-06bc-a15f-f6bd-6cdca1f6dd56
rubriek                                                       358
notaris                                               JAN VERLEIJ
inventarisNr                                       

In [112]:
jan_verleij.to_csv('clean_data.csv', index=False)

## VOC Opvarende Lijst

In [4]:
voc = pd.read_csv('../vocclustered.csv', sep='	')

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (17,27,35,36,38) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
for column in voc.columns:
    print(voc.iloc[0][column], column)

104944 VOCOP_id
Adriaen van Renteregem fullNameOriginal
Adriaen firstNameOriginal
nan patronymicOriginal
van familyNamePrefixOriginal
Renteregem familyNameOriginal
Wassenaar placeOfOrigin
Wassenaar NL toponymCountryCode
NL countryCode
http://sws.geonames.org/2744991/ URI
52.14583 LAT
4.40278 LON
nan country/region
7.0 wageMin
Sailor rank
11.0 wageMax
7.0 wageStandard_DASminSupplementedBeini
98140.0 HISCO_CODE
https://iisg.amsterdam/hisco/code/hisco/98140 HISCO_URI
1700-05-05 dateBeginService
1704-08-11 dateEndServiceNoZeros
Repatriated reasonEndService
Concordia endServiceWhere
Huis te Loo shipOutward
DAS_ship0682 shipID
875.0 shipTonnage
92895.0 voyageID
1835.4 voyNumberDAS
Ja monthLetter
Ja debtLetter
nan generalRemark
nan boardedAtCape
nan boardedAtCapeShipName
nan boardedAtCapeChamber
nan boardedAtCapeVoyageID
nan boardedAtCapeVoyNumberDAS
nan boardedAtCapeShipID
CONCORDIA shipReturn
6107.2 DASvoyageNumberReturnJourney
96993.0 DASvoyageReturnID
DAS_ship0259 DASshipReturnID
900.0 DA

In [12]:
clean = pd.read_csv('../clean_data.csv')
name_list = []
for z in notebook.tqdm(clean.namen):
    for x in ast.literal_eval(z):
        if x['tussenvoegsel'] != None:
            name_list.append(x['voornaam'] + " " + x['tussenvoegsel'] + " " + x['achternaam'])
        elif x['voornaam'] and x['achternaam'] != None:
            name_list.append(x['voornaam'] + " " + x['achternaam'])

In [13]:
len(name_list)

65433

In [18]:
holder = []
for x in notebook.tqdm(name_list):
    if x in voc.fullNameOriginal.tolist():
        holder.append(x)


KeyboardInterrupt: 

In [19]:
test = voc[(voc.fullNameOriginal.isin(holder)) & (voc.in_clustered_set == 1) & (voc.date_begin_service_complete.isin(clean.datering))][['fullNameOriginal', 'rank',
'date_begin_service_complete',
'date_end_service_complete', 'shipOutward', 'shipReturn',
'cluster_ID',
'linked_vocop_id',
'HISCO_CODE',
'LON',
'LAT',
'countryCode',
'placeOfOrigin',
'reasonEndService',
'in_clustered_set', 
'endServiceWhere']]

In [20]:
name_list = []
# for index, row in notebook.tqdm(test.iterrows()):
for indeks, rij in clean.iterrows():
    for x in ast.literal_eval(rij.namen):
        if x['tussenvoegsel'] != None:
            naam = (x['voornaam'] + " " + x['tussenvoegsel'] + " " + x['achternaam'])
        elif x['voornaam'] and x['achternaam'] != None:
            naam = (x['voornaam'] + " " + x['achternaam'])
        if naam == 'Cornelis Vis':
            print(indeks)
#         if row.shipOutward in str(rij.beschrijving):
#             print(index, indeks, row.shipOutward)
                
                

10848


In [51]:
voc.iloc[66979]['placeOfOrigin']

'Frederikstad'

In [44]:
for index, x in voc[(voc.shipReturn.str.lower() == 'KRABBENDIJKE'.lower())].iterrows():
    print(x)

VOCOP_id                                      235105
fullNameOriginal                      Pieter Schanen
firstNameOriginal                             Pieter
patronymicOriginal                               NaN
familyNamePrefixOriginal                         NaN
                                       ...          
date_begin_service_complete               1731-05-11
date_end_service_complete                 1748-09-02
cape_boarder_is_linked                           NaN
linked_vocop_id                                  NaN
cape_boarder_rule              no_cape_boarder_issue
Name: 43763, Length: 67, dtype: object
VOCOP_id                                      243250
fullNameOriginal                   Hieronimus Looman
firstNameOriginal                         Hieronimus
patronymicOriginal                               NaN
familyNamePrefixOriginal                         NaN
                                       ...          
date_begin_service_complete               1734-03-12
date_en

VOCOP_id                                      659885
fullNameOriginal                     Henderik Claret
firstNameOriginal                           Henderik
patronymicOriginal                               NaN
familyNamePrefixOriginal                         NaN
                                       ...          
date_begin_service_complete               1735-06-01
date_end_service_complete                 1745-08-24
cape_boarder_is_linked                           NaN
linked_vocop_id                                  NaN
cape_boarder_rule              no_cape_boarder_issue
Name: 261601, Length: 67, dtype: object
VOCOP_id                                      660888
fullNameOriginal                       Hendrik Creps
firstNameOriginal                            Hendrik
patronymicOriginal                               NaN
familyNamePrefixOriginal                         NaN
                                       ...          
date_begin_service_complete               1737-03-10
date_e

VOCOP_id                                     1065727
fullNameOriginal                     Andries Swerisz
firstNameOriginal                            Andries
patronymicOriginal                               NaN
familyNamePrefixOriginal                         NaN
                                       ...          
date_begin_service_complete               1752-01-12
date_end_service_complete                 1753-08-09
cape_boarder_is_linked                           NaN
linked_vocop_id                                  NaN
cape_boarder_rule              no_cape_boarder_issue
Name: 347157, Length: 67, dtype: object
VOCOP_id                                     1065729
fullNameOriginal                        Jacob Schols
firstNameOriginal                              Jacob
patronymicOriginal                               NaN
familyNamePrefixOriginal                         NaN
                                       ...          
date_begin_service_complete               1752-01-12
date_e

VOCOP_id                                     1183173
fullNameOriginal                          Jean Maree
firstNameOriginal                               Jean
patronymicOriginal                               NaN
familyNamePrefixOriginal                         NaN
                                       ...          
date_begin_service_complete               1749-09-30
date_end_service_complete                 1751-06-10
cape_boarder_is_linked                           NaN
linked_vocop_id                                  NaN
cape_boarder_rule              no_cape_boarder_issue
Name: 463698, Length: 67, dtype: object
VOCOP_id                                     1183770
fullNameOriginal                          Jacob Berg
firstNameOriginal                              Jacob
patronymicOriginal                               NaN
familyNamePrefixOriginal                         NaN
                                       ...          
date_begin_service_complete               1746-05-07
date_e

In [511]:
for x in ast.literal_eval(clean.iloc[10848].namen):
        if x['tussenvoegsel'] != None:
            print(x['voornaam'] + " " + x['tussenvoegsel'] + " " + x['achternaam'])
        elif x['voornaam'] and x['achternaam'] != None:
            print(x['voornaam'] + " " + x['achternaam'])


Cornelis Vis
Jan Govertsz


In [9]:
clean.iloc[10848]

uuid                         da6f9044-1588-3bf2-1e2f-72b7b374e3db
rubriek                                                       358
notaris                                               JAN VERLEIJ
inventarisNr                                                11905
akteNr                                                      18207
akteType                                               Machtiging
datering                                               1748-09-19
taal                                                   nederlands
beschrijving    \nschip 't Huys te Spijk, VOC kamer Zeeland; s...
namen           [{'voornaam': 'Cornelis', 'tussenvoegsel': Non...
urls               ['KLAB05559000102.JPG', 'KLAB05559000103.JPG']
text            No: 259 Procuratie gepoassseert den 19e: Septe...
Name: 10848, dtype: object

In [496]:
for x, y in clean.iterrows():
    if 'KRABBENDIJKE' in str(y.beschrijving).upper():
        print(x)

10846


In [203]:
2152
4379
8492
8364
2036

'Nieuw Vijvervreugd'

In [23]:
voc.iloc[66931].shipReturn

'KRABBENDIJKE'

In [21]:
test = pd.read_csv('../clean_data.csv')

In [24]:
test.drop(['taal', 'akteNr', 'inventarisNr'], axis=1)

uuid  rubriek      notaris  \
0      17599c0c-3305-165c-aae3-eddbb497e4b4      358  JAN VERLEIJ   
1      1b037028-bea5-0fa8-e6c5-68155f5f2b21      358  JAN VERLEIJ   
2      3d072c4e-8cf9-daf8-878d-a64133360fc8      358  JAN VERLEIJ   
3      3ddb90b2-16e5-8f87-fda9-1bfb2804bb09      358  JAN VERLEIJ   
4      412e645c-db5d-673e-5fce-006cfab5d7f1      358  JAN VERLEIJ   
...                                     ...      ...          ...   
13058  dcc03903-6c44-8ed3-2e3e-4539f500958c      358  JAN VERLEIJ   
13059  e9390ff6-153c-2679-e855-53d67352ea9c      358  JAN VERLEIJ   
13060  ec3d450b-486e-7e87-8894-adfd36d5f544      358  JAN VERLEIJ   
13061  fe9f2abb-f67c-f214-23ec-41d724ed5ce7      358  JAN VERLEIJ   
13062  c5c2883c-7e16-b99f-4417-bb1d36d8f691      358  JAN VERLEIJ   

                akteType    datering  \
0              Testament  1741-03-16   
1             Attestatie  1741-07-26   
2             Insinuatie  1741-09-27   
3             Attestatie  1741-09-08   
4              Testament  1741-12-29   
...                  ...         ...   
13058          Kwitantie  1770-01-17   
13059         Machtiging  1769-03-22   
13060          Kwitantie  1770-08-11   
13061  Scheepsverklaring  1769-02-22   
13062             Beraad  1775-01-18   

                                            beschrijving  \
0                                                    NaN   
1                               \nordentelijke vrijage\n   
2                                    \nschuld opgeeist\n   
3      \naantasting goede naam na overnachtingen na e...   
4                                                    NaN   
...                                                  ...   
13058     \nKwitantie voor ontvangst van nalatenschap.\n   
13059  \nVOC schip de Jonge Samuel. betaling uit gage...   
13060                                                NaN   
13061  \nSchip des Jonge Fontijns heeft vertraging op...   
13062                                                NaN   

                                                   namen  \
0      [{'voornaam': 'Trijntje', 'tussenvoegsel': Non...   
1      [{'voornaam': 'Gerard', 'tussenvoegsel': None,...   
2      [{'voornaam': 'Dirk', 'tussenvoegsel': 'van de...   
3      [{'voornaam': 'Sara', 'tussenvoegsel': None, '...   
4      [{'voornaam': 'Aaffje Jans', 'tussenvoegsel': ...   
...                                                  ...   
13058  [{'voornaam': 'Theodorus', 'tussenvoegsel': 'v...   
13059  [{'voornaam': 'Pieter', 'tussenvoegsel': 'van'...   
13060  [{'voornaam': 'Annitje', 'tussenvoegsel': None...   
13061  [{'voornaam': 'Jelle', 'tussenvoegsel': None, ...   
13062  [{'voornaam': 'Jan', 'tussenvoegsel': None, 'a...   

                                                    urls  \
0      ['KLAB05372000012.JPG', 'KLAB05372000013.JPG',...   
1      ['KLAB05372000193.JPG', 'KLAB05372000194.JPG',...   
2      ['KLAB05372000281.JPG', 'KLAB05372000282.JPG',...   
3      ['KLAB05372000244.JPG', 'KLAB05372000245.JPG',...   
4      ['KLAB05372000380.JPG', 'KLAB05372000381.JPG',...   
...                                                  ...   
13058     ['KLAB07041000258.JPG', 'KLAB07041000259.JPG']   
13059  ['KLAB07041000174.JPG', 'KLAB07041000175.JPG',...   
13060  ['KLAB07041000307.JPG', 'KLAB07041000308.JPG',...   
13061  ['KLAB07041000100.JPG', 'KLAB07041000101.JPG',...   
13062  ['KLAB07039000033.JPG', 'KLAB07039000034.JPG',...   

                                                    text  
0      N: 4 1e de Testateuren hebben verklaerd te Sam...  
1      Verklaing gepasseert den 26 Julij 1741No: 60- ...  
2      Insinuatie gedaan den 27e: September 1741No: 8...  
3      Verklaring gepasseert den 8e: Septembr: 141No:...  
4      No: 121 Janlron de Testateuren te Samen benede...  
...                                                  ...  
13058  No: 1 Quitantie gecasseerd den 17e: Januarij 1...  
13059  No: 51 Procuratie gecasseerd den 22e: Maart 17...  
13060  No: 11 Quitantie gecasseerd 